In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
import os

# --- Cargar modelo ---
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHECKPOINT_PATH = "F:/IA/SAM2/checkpoints/sam2.1_hiera_large.pt"
CONFIG_PATH = "F:/IA/SAM2/segment-anything-2/sam2/configs/sam2.1_hiera_l.yaml"

sam2_model = build_sam2(CONFIG_PATH, CHECKPOINT_PATH, device=DEVICE)
predictor = SAM2ImagePredictor(sam2_model)

print("✅ Modelo cargado.")

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output

image = None
points = []
labels = []

def load_image():
    global image
    uploader = widgets.FileUpload(accept='image/*', multiple=False)
    display(uploader)
    
    def on_upload(change):
        global image
        for name, file_info in uploader.value.items():
            image = Image.open(file_info['content']).convert("RGB")
            plt.figure(figsize=(12, 8))
            plt.imshow(image)
            plt.axis('off')
            plt.show()
            break
    
    uploader.observe(on_upload, names='value')

load_image()

In [ ]:
def on_click(event):
    global points, labels, image
    if image is None:
        print("⚠️ Sube una imagen primero.")
        return
    
    if event.inaxes is not None:
        x, y = int(event.xdata), int(event.ydata)
        points.append([x, y])
        labels.append(1)  # 1 = foreground
        
        # Convertir a numpy
        image_np = np.array(image)
        predictor.set_image(image_np)
        
        # Predecir máscara
        input_points = np.array(points)
        input_labels = np.array(labels)
        
        masks, scores, _ = predictor.predict(
            point_coords=input_points,
            point_labels=input_labels,
            multimask_output=False,
        )
        
        mask = masks[0]
        
        # Superponer máscara
        overlay = image_np.copy()
        overlay[mask] = [255, 0, 0]  # Rojo
        blended = (image_np * 0.6 + overlay * 0.4).astype(np.uint8)
        
        # Mostrar
        clear_output(wait=True)
        plt.figure(figsize=(12, 8))
        plt.imshow(blended)
        plt.axis('off')
        plt.title("Haz clic para añadir más puntos. Usa Shift + clic para excluir.")
        plt.show()
        
        # Guardar máscara
        mask_pil = Image.fromarray((mask * 255).astype(np.uint8))
        mask_pil.save("mask_output.png")
        print("✅ Máscara guardada como 'mask_output.png'")

fig = plt.figure(figsize=(12, 8))
cid = fig.canvas.mpl_connect('button_press_event', on_click)
plt.title("Haz clic en la imagen para segmentar. Usa Shift + clic para excluir.")
plt.show()